In [0]:
%pip install snowflake
%pip install snowflake-connector-python
import pandas as pd
from pyspark.sql import SparkSession
import snowflake.connector

# Snowflake connection parameters
account = 'My Account'
user = 'My User Name'
password = 'My Password'
warehouse = 'My Warehouse Name'
database = 'My Database Name'
schema = 'My Schema Name'
table_name = 'My Table Name'  # Table name without '.csv' suffix

# Snowflake URL
snowflake_url = ''

# Load CSV into a Spark DataFrame
file_path = "/FileStore/Details.csv"
spark = SparkSession.builder.appName("SnowflakeIntegration").getOrCreate()
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Snowflake connection
conn = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema,
    sfURL=snowflake_url  # Include Snowflake URL
)


# Create cursor
cur = conn.cursor()

# Replace spaces and dots in table name
table_name_sanitized = table_name.replace(' ', '_').replace('.', '_')

# Create table in Snowflake with dynamically generated column definitions
column_definitions = ', '.join([f'"{col}" STRING' for col in df.columns])
cur.execute(f'CREATE OR REPLACE TABLE "{schema}"."{table_name_sanitized}" ({column_definitions})')

# Write DataFrame to Snowflake table
df.write.format("snowflake") \
    .option("sfURL", snowflake_url) \
    .option("sfUser", user) \
    .option("sfPassword", password) \
    .option("sfDatabase", database) \
    .option("sfSchema", schema) \
    .option("sfWarehouse", warehouse) \
    .option("dbtable", table_name_sanitized) \
    .mode("overwrite") \
    .save()

# Commit the transaction
conn.commit()

# Close cursor and connection
cur.close()
conn.close()


Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
